In [1]:
import requests
import urllib.parse
import xml.etree.ElementTree as ET
from dotenv import find_dotenv, load_dotenv
import os
import math
import re
import string
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pickle
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /home/shalini_tiwari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
os.environ["OPENAI_API_KEY"] = "f053125d3240ecfff2759eb6c57a0d1b"

In [6]:
#Variable Declaration
dois = []
search_query = 'subject:"Law"'
doi_count_per_request = 1000
desired_doi_count = 1000#math.inf
doi_request_url = "https://api.springernature.com/meta/v2/dois"

In [7]:
def get_response(request_url, params, headers, debug=True):
    response = requests.get(request_url, params=params, headers=headers)
    if debug:
        print(f"Request : {response.url}")
        print(f"Response: {response.status_code}: {response.reason}")
    if response.status_code != 200:        
        return None
    else:
        return response.text
    



In [8]:
def extract_dois(root):
    dois = []
    for doi in root.iter('doi'):
        dois.append(doi.text)
    return dois

In [9]:
def get_total_doi_count(root):
    result_tag = root.find("result")
    total_available_doi_count = int(result_tag.find("total").text)
    return total_available_doi_count

In [10]:
def get_retrieved_doi_count(root):
    result_tag = root.find("result")
    retrieved_doi_count = int(result_tag.find("pageLength").text)
    return retrieved_doi_count

In [11]:
payload = {
            "q": search_query,
            "api_key": "f053125d3240ecfff2759eb6c57a0d1b", 
            "s": 0, 
            "p": doi_count_per_request
          }
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
params = urllib.parse.urlencode(payload, safe=':+')

In [12]:
total_retrieved_doi_count = 0
if math.isfinite(desired_doi_count):
    iterations = math.ceil(desired_doi_count/doi_count_per_request)
else:
    iterations = math.inf
iternum = 0

In [13]:
int(total_retrieved_doi_count/doi_count_per_request)
payload["s"] = total_retrieved_doi_count + 1
params = urllib.parse.urlencode(payload, safe=':+')
print(f"Total api requests: {iterations}")

Total api requests: 1


In [14]:
while True:
    response = get_response(doi_request_url, params, headers)
    if response == None:
        print("Error: No response received")
        break
    root = ET.fromstring(response)
    retrieved_doi_count = get_retrieved_doi_count(root)
    total_retrieved_doi_count += retrieved_doi_count
    total_doi_count = get_total_doi_count(root)
    dois.extend(extract_dois(root))
    iternum += 1
    if iternum >= iterations or total_retrieved_doi_count >= total_doi_count:
        break
    payload["s"] = (iternum * doi_count_per_request) + 1
    params = urllib.parse.urlencode(payload, safe=':+')

Request : https://api.springernature.com/meta/v2/dois?q=subject:%22Law%22&api_key=f053125d3240ecfff2759eb6c57a0d1b&s=1&p=1000
Response: 200: OK


In [15]:
total_doi_count

89034

In [16]:
dois

['10.1007/s11196-023-10030-1',
 '10.1007/s11572-023-09697-1',
 '10.1007/s11572-023-09696-2',
 '10.1007/s10991-023-09328-8',
 '10.1007/s40803-023-00193-1',
 '10.1007/s40318-023-00246-5',
 '10.1007/s11196-023-10033-y',
 '10.1007/s00350-023-6534-5',
 '10.1007/s40319-023-01370-3',
 '10.1007/s11196-023-09990-1',
 '10.1007/s00350-023-6538-1',
 '10.1007/s40319-023-01371-2',
 '10.1007/s40318-023-00243-8',
 '10.1007/s11196-023-10001-6',
 '10.1007/s00350-023-6540-7',
 '10.1007/s00350-023-6544-3',
 '10.1007/s11196-023-09992-z',
 '10.1007/s00350-023-6548-z',
 '10.1007/s10991-023-09330-0',
 '10.1007/s00350-023-6550-5',
 '10.1007/s11196-023-09993-y',
 '10.1007/s11196-022-09942-1',
 '10.1007/s10991-023-09331-z',
 '10.1007/s10991-023-09332-y',
 '10.1007/s11196-023-10008-z',
 '10.1007/s10991-023-09333-x',
 '10.1007/s40804-023-00295-1',
 '10.1007/s00350-023-6529-2',
 '10.1007/s10991-023-09334-w',
 '10.1007/s11196-023-09997-8',
 '10.1007/s11196-022-09946-x',
 '10.1007/s00350-023-6531-8',
 '10.1007/s10991

In [17]:
with open("dois.txt", 'w') as f:
    for d in dois:
        f.write(str(d) + '\n')